In [1]:
! pip install talib-binary
! gdown 17OYeD6dlfwX28FNoUpqXkLVpzjUtTCRW
! unzip /content/ThaiDepression.zip
! pip install -qqq greykite
! pip install --upgrade pandas
! python -m pip uninstall matplotlib
! pip install matplotlib==3.1.3
! pip install tsai
! pip install sktime
! gdown 1h6z8hj_4cXAZ-qtssjWIEga6zcDyfP1b

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.4 MB 2.0 MB/s 
Downloading...
From: https://drive.google.com/uc?id=17OYeD6dlfwX28FNoUpqXkLVpzjUtTCRW
To: /content/ThaiDepression.zip
100% 8.78M/8.78M [00:00<00:00, 49.0MB/s]
Archive:  /content/ThaiDepression.zip
   creating: ThaiDepression/2016/
  inflating: ThaiDepression/2016/ahb-2016-10.csv  
  inflating: ThaiDepression/2016/ahb-2016-10.pdf  
  inflating: ThaiDepression/2016/ahb-2016-11.csv  
  inflating: ThaiDepression/2016/ahb-2016-11.pdf  
  inflating: ThaiDepression/2016/ahb-2016-12.csv  
  inflating: ThaiDepression/2016/ahb-2016-12.pdf  
   creating: ThaiDepression/2017/
  inflating: ThaiDepression/2017/ahb-2017-01.csv  
  inflating: ThaiDepression/2017/ahb-2017-01.pdf  
  inflating: ThaiDepression/2017/ahb-2017-02.csv  
  inflating: ThaiDepression/2017/ahb-2017-02.pdf  
  inflating: ThaiDepression/2017/ahb-2017-03.csv  
  inflating: ThaiD

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 241 kB 2.1 MB/s 
     |████████████████████████████████| 750.6 MB 9.6 kB/s 
     |████████████████████████████████| 2.5 MB 57.1 MB/s 
     |████████████████████████████████| 19.1 MB 30.8 MB/s 
     |████████████████████████████████| 19.1 MB 29.7 MB/s 
     |████████████████████████████████| 21.0 MB 23.0 MB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.12.1+cu113
    Uninstalling torch-1.12.1+cu113:
      Successfully uninstalled torch-1.12.1+cu113
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.13.1+cu113
    Uninstalling torchvision-0.13.1+cu113:
      Successfully uninstalled torchvision-0.13.1+cu113
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchtext 0.1

In [2]:
import os
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import talib
from greykite.framework.templates.autogen.forecast_config import ForecastConfig
from greykite.framework.templates.autogen.forecast_config import MetadataParam
from greykite.framework.templates.forecaster import Forecaster 
from greykite.framework.templates.model_templates import ModelTemplateEnum
from greykite.framework.utils.result_summary import summarize_grid_search_results
import plotly
import warnings
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import  plot_pacf
from statsmodels.tsa.stattools import pacf
import itertools
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.model_selection import train_test_split
from tsai.all import *
from tsai.inference import load_learner
from pandas import Series
from sklearn.preprocessing import MinMaxScaler
from pmdarima import arima
from sktime.forecasting.arima import AutoARIMA
from sktime.forecasting.model_selection import ExpandingWindowSplitter
from sktime.forecasting.model_evaluation import evaluate
from sktime.utils.plotting import plot_series
import seaborn as sns
from sklearn import preprocessing
from IPython.display import display
from fastai.imports import *
from sklearn import metrics
from pandas.tseries.offsets import DateOffset
import math
from sklearn.impute import SimpleImputer
import torch
from sklearn.model_selection import TimeSeriesSplit
import json
from torch.utils.data import TensorDataset, DataLoader
import torch.optim as optim                       

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"{device}" " is available.")

cpu is available.


#Data Preparation

In [3]:
path = "/content/ThaiDepression/"
year_list = os.listdir(path)
year_list.sort()

df_list = []
name_list = []
 
for i in range(len(year_list)):
    file_list = os.listdir(path+year_list[i])
    file_list.sort()
    for f in file_list:
        if '.csv' in f:
            # print(f)
            name_list.append(f[4:11])
            df_list.append(pd.read_csv(path+year_list[i]+'/'+f, header=None, index_col=False))


finished_process = []
for i in range(len(df_list[0])):
    finished_process.append(pd.DataFrame(df_list[0].iloc[i,:]).T.reset_index(drop=True))
    finished_process[i]['y-m'] = name_list[0]


i=1
while(i != len(df_list)):
    # print('i=' + str(i))
    if len(df_list[i]) == 89:
        insert = { 0 : np.nan, 1 : np.nan, 2 : np.nan, 3 : np.nan, 4 : np.nan, 5 : np.nan, 6 :np.nan, 7 : np.nan, 8 : np.nan, 9 : np.nan, 'y-m' : name_list[i]}
        finished_process[89] = pd.concat([finished_process[89], pd.DataFrame(insert, index=[0])], ignore_index = True)
    for j in range(len(df_list[i])):
        # print('j=' + str(j))
        if len(df_list[i].columns) == 6 :
            insert = { 0 : df_list[i].iloc[j,0], 1 : df_list[i].iloc[j,1], 2 : np.nan, 3 : df_list[i].iloc[j,2], 4 : df_list[i].iloc[j,3], 5 : np.nan, 6 :np.nan, 7 : df_list[i].iloc[j,4], 8 : np.nan, 9 : df_list[i].iloc[j,5], 'y-m' : name_list[i]}
            finished_process[j] = pd.concat([finished_process[j], pd.DataFrame(insert, index=[0])], ignore_index = True)
        else:
            finished_process[j] = pd.concat([finished_process[j], pd.DataFrame(df_list[i].iloc[j,:]).T.reset_index(drop=True)], ignore_index = True)
            finished_process[j]['y-m'][i] = name_list[i]
    i=i+1


for i in range(len(finished_process)):
    finished_process[i].index = finished_process[i]['y-m'].values
    finished_process[i].drop(['y-m'], axis=1, inplace=True)
    # finished_process[i].index = pd.to_datetime(finished_process[i].index, format='%Y-%m')
    finished_process[i] = finished_process[i].replace(',','', regex=True)  
    finished_process[i] = finished_process[i].apply(pd.to_numeric)

    # pct_change(1) = (value on this index - value on previous index) / value on previous index

    finished_process[i]['0_pct_change'] = finished_process[i][0].pct_change(1)
    finished_process[i]['1_pct_change'] = finished_process[i][1].pct_change(1)
    finished_process[i]['2_pct_change'] = finished_process[i][2].pct_change(1)
    finished_process[i]['3_pct_change'] = finished_process[i][3].pct_change(1)
    finished_process[i]['4_pct_change'] = finished_process[i][4].pct_change(1)
    finished_process[i]['5_pct_change'] = finished_process[i][5].pct_change(1)
    finished_process[i]['6_pct_change'] = finished_process[i][6].pct_change(1)
    finished_process[i]['7_pct_change'] = finished_process[i][7].pct_change(1)
    finished_process[i]['8_pct_change'] = finished_process[i][8].pct_change(1)
    finished_process[i]['9_pct_change'] = finished_process[i][9].pct_change(1)
    # finished_process[i].loc[finished_process[i][3] == np.nan, '3_pct_change'] = np.nan
    # finished_process[i]['3_pct_change'] = finished_process[i][3].apply(lambda x: np.nan if x == np.nan else x)


finished_process[89][3] = pd.read_csv('/content/AHB_13.csv', index_col=0)['Bangkok']

In [4]:
AHB_data = []
AHB_list = [9, 15, 21, 30, 39, 48, 53, 61, 66, 72, 80, 88, 89]
province_AHB_list = [
            [0, 1, 2, 3, 4, 5, 6, 7, 8],
            [0, 10, 11, 12, 13, 14],
            [0, 16, 17, 18, 19, 20],
            [0, 22, 23, 24, 25, 26, 27, 28, 29],
            [0, 31, 32, 33, 34, 35, 36, 37, 38],
            [0, 40, 41, 42, 43, 44, 45, 46, 47],
            [0, 49, 50, 51, 52],
            [0, 54, 55, 56, 57, 58, 59, 60],
            [0, 62, 63, 64, 65],
            [0, 67, 68, 69, 70, 71],
            [0, 73, 74, 75, 76, 77, 78, 79],
            [0, 81, 82, 83, 84, 85, 86, 87],
            [0, 89]
            ]
# AHB
for j in range(len(AHB_list)):
    AHB = pd.DataFrame(finished_process[AHB_list[j]][3].values, columns=['AHB_'+str(j+1)])
    for i in range(len(province_AHB_list[j])):
        if i == 0:
            continue
        AHB = pd.concat([AHB, finished_process[province_AHB_list[j][i]][3].reset_index(drop='index')], axis = 1)
    temp = [str(x) for x in province_AHB_list[j]]
    temp[0] = 'AHB_'+str(j+1)
    AHB.columns = temp
    AHB['y'] = AHB['AHB_'+str(j+1)].shift(-1)
    AHB.index = finished_process[9][3].index
    AHB.drop(AHB.tail(1).index,inplace=True)
    AHB_data.append(AHB)


#normalized_AHB_data = []
#for i in range(len(AHB_data)):
#    temp = (AHB_data[i]-AHB_data[i].mean())/AHB_data[i].std()
    # temp = (AHB_data[i]-AHB_data[i].min())/(AHB_data[i].max()-AHB_data[i].min())
#    normalized_AHB_data.append(temp)

In [ ]:
AHB_data

[          AHB_1      1     2      3     4      5     6      7      8         y
 2016-10   53656  10383  5315   4266  4057  15408  2002   9011   3214   53653.0
 2016-11   53653  10383  5315   4265  4057  15406  2002   9011   3214   53650.0
 2016-12   53650  10382  5315   4265  4057  15404  2002   9011   3214   53978.0
 2017-01   53978  10382  5655   4265  4057  15393  2002   9011   3213   53976.0
 2017-02   53976  10382  5655   4264  4057  15392  2002   9011   3213   57981.0
 ...         ...    ...   ...    ...   ...    ...   ...    ...    ...       ...
 2022-02  137049  25005  9671  10674  7947  47073  6876  17426  12377  137869.0
 2022-03  137869  25223  9711  10788  7988  47292  6924  17444  12499  139544.0
 2022-04  139544  25456  9781  10932  8095  48020  7004  17606  12650  140152.0
 2022-05  140152  25619  9833  10977  8122  48155  7026  17703  12717  141246.0
 2022-06  141246  25836  9884  11069  8161  48616  7090  17783  12807  142732.0
 
 [69 rows x 10 columns],
          AHB

#Deep Learning

In [5]:
import torch.nn as nn

class RNNModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, layer_dim, output_dim, dropout_prob):
        super(RNNModel, self).__init__()

        # Defining the number of layers and the nodes in each layer
        self.hidden_dim = hidden_dim
        self.layer_dim = layer_dim

        # RNN layers
        self.rnn = nn.RNN(
            input_dim, hidden_dim, layer_dim, batch_first=True, dropout=dropout_prob
        )
        # Fully connected layer
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        # Initializing hidden state for first input with zeros
        h0 = torch.zeros(self.layer_dim, x.size(0), self.hidden_dim).requires_grad_()

        # Forward propagation by passing in the input and hidden state into the model
        out, h0 = self.rnn(x, h0.detach())

        # Reshaping the outputs in the shape of (batch_size, seq_length, hidden_size)
        # so that it can fit into the fully connected layer
        out = out[:, -1, :]

        # Convert the final state to our desired output shape (batch_size, output_dim)
        out = self.fc(out)
        return out

In [6]:
class LSTMModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, layer_dim, output_dim, dropout_prob):
        super(LSTMModel, self).__init__()

        # Defining the number of layers and the nodes in each layer
        self.hidden_dim = hidden_dim
        self.layer_dim = layer_dim

        # LSTM layers
        self.lstm = nn.LSTM(
            input_dim, hidden_dim, layer_dim, batch_first=True, dropout=dropout_prob
        )

        # Fully connected layer
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        # Initializing hidden state for first input with zeros
        h0 = torch.zeros(self.layer_dim, x.size(0), self.hidden_dim).requires_grad_()

        # Initializing cell state for first input with zeros
        c0 = torch.zeros(self.layer_dim, x.size(0), self.hidden_dim).requires_grad_()

        # We need to detach as we are doing truncated backpropagation through time (BPTT)
        # If we don't, we'll backprop all the way to the start even after going through another batch
        # Forward propagation by passing in the input, hidden state, and cell state into the model
        out, (hn, cn) = self.lstm(x, (h0.detach(), c0.detach()))

        # Reshaping the outputs in the shape of (batch_size, seq_length, hidden_size)
        # so that it can fit into the fully connected layer
        out = out[:, -1, :]

        # Convert the final state to our desired output shape (batch_size, output_dim)
        out = self.fc(out)

        return out

In [7]:
class GRUModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, layer_dim, output_dim, dropout_prob):
        super(GRUModel, self).__init__()

        # Defining the number of layers and the nodes in each layer
        self.layer_dim = layer_dim
        self.hidden_dim = hidden_dim

        # GRU layers
        self.gru = nn.GRU(
            input_dim, hidden_dim, layer_dim, batch_first=True, dropout=dropout_prob
        )

        # Fully connected layer
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        # Initializing hidden state for first input with zeros
        h0 = torch.zeros(self.layer_dim, x.size(0), self.hidden_dim).requires_grad_()

        # Forward propagation by passing in the input and hidden state into the model
        out, _ = self.gru(x, h0.detach())

        # Reshaping the outputs in the shape of (batch_size, seq_length, hidden_size)
        # so that it can fit into the fully connected layer
        out = out[:, -1, :]

        # Convert the final state to our desired output shape (batch_size, output_dim)
        out = self.fc(out)

        return out

In [8]:
def get_model(model, model_params):
    models = {
        "rnn": RNNModel,
        "lstm": LSTMModel,
        "gru": GRUModel,
    }
    return models.get(model.lower())(**model_params)

In [9]:
class Optimization:
    """Optimization is a helper class that allows training, validation, prediction.

    Optimization is a helper class that takes model, loss function, optimizer function
    learning scheduler (optional), early stopping (optional) as inputs. In return, it
    provides a framework to train and validate the models, and to predict future values
    based on the models.

    Attributes:
        model (RNNModel, LSTMModel, GRUModel): Model class created for the type of RNN
        loss_fn (torch.nn.modules.Loss): Loss function to calculate the losses
        optimizer (torch.optim.Optimizer): Optimizer function to optimize the loss function
        train_losses (list[float]): The loss values from the training
        val_losses (list[float]): The loss values from the validation
        last_epoch (int): The number of epochs that the models is trained
    """
    def __init__(self, model, loss_fn, optimizer):
        """
        Args:
            model (RNNModel, LSTMModel, GRUModel): Model class created for the type of RNN
            loss_fn (torch.nn.modules.Loss): Loss function to calculate the losses
            optimizer (torch.optim.Optimizer): Optimizer function to optimize the loss function
        """
        self.model = model
        self.loss_fn = loss_fn
        self.optimizer = optimizer
        self.train_losses = []
        self.val_losses = []
        self.last_epoch = 0
        
    def train_step(self, x, y):
        """The method train_step completes one step of training.

        Given the features (x) and the target values (y) tensors, the method completes
        one step of the training. First, it activates the train mode to enable back prop.
        After generating predicted values (yhat) by doing forward propagation, it calculates
        the losses by using the loss function. Then, it computes the gradients by doing
        back propagation and updates the weights by calling step() function.

        Args:
            x (torch.Tensor): Tensor for features to train one step
            y (torch.Tensor): Tensor for target values to calculate losses

        """
        # Sets model to train mode
        self.model.train()

        # Makes predictions
        yhat = self.model(x)

        # Computes loss
        loss = self.loss_fn(y, yhat)

        # Computes gradients
        loss.backward()

        # Updates parameters and zeroes gradients
        self.optimizer.step()
        self.optimizer.zero_grad()

        # Returns the loss
        return loss.item()

    def train(self, train_loader, val_loader, batch_size=64, n_epochs=50, n_features=1, the_last_loss = 100, patience = 2, trigger_times = 0):
        """The method train performs the model training

        The method takes DataLoaders for training and validation datasets, batch size for
        mini-batch training, number of epochs to train, and number of features as inputs.
        Then, it carries out the training by iteratively calling the method train_step for
        n_epochs times. If early stopping is enabled, then it  checks the stopping condition
        to decide whether the training needs to halt before n_epochs steps. Finally, it saves
        the model in a designated file path.

        Args:
            train_loader (torch.utils.data.DataLoader): DataLoader that stores training data
            val_loader (torch.utils.data.DataLoader): DataLoader that stores validation data
            batch_size (int): Batch size for mini-batch training
            n_epochs (int): Number of epochs, i.e., train steps, to train
            n_features (int): Number of feature columns

        """

        model_path = f'{self.model}_{datetime.now().strftime("%Y-%m-%d %H:%M:%S")}'

        for epoch in range(1, n_epochs + 1):
            batch_losses = []
            for x_batch, y_batch in train_loader:
                x_batch = x_batch.view([batch_size, -1, n_features]).to(device)
                y_batch = y_batch.to(device)
                loss = self.train_step(x_batch, y_batch)
                batch_losses.append(loss)
            training_loss = np.mean(batch_losses)
            self.train_losses.append(training_loss)

            with torch.no_grad():
                batch_val_losses = []
                for x_val, y_val in val_loader:
                    x_val = x_val.view([batch_size, -1, n_features]).to(device)
                    y_val = y_val.to(device)
                    self.model.eval()
                    yhat = self.model(x_val)
                    val_loss = self.loss_fn(y_val, yhat).item()
                    batch_val_losses.append(val_loss)
                validation_loss = np.mean(batch_val_losses)


                current_loss = validation_loss
                #print('The Current Loss:', current_loss)

                if current_loss > the_last_loss:
                    trigger_times = trigger_times + 1
                    #print('Trigger Times:', trigger_times)

                    if trigger_times >= patience:
                        #print('Early Stopping!')
                        self.val_losses.append(validation_loss)
                        self.last_epoch = epoch 
                        break


                else:
                    #print('Trigger Times: 0')
                    trigger_times = 0
                the_last_loss = current_loss
                self.val_losses.append(validation_loss)
            self.last_epoch = epoch 

            #if (epoch <= 10) | (epoch % 50 == 0):
            #    print(
            #        f"[{epoch}/{n_epochs}] Training loss: {training_loss:.4f}\t Validation loss: {validation_loss:.4f}"
            #    )

        #torch.save(self.model.state_dict(), model_path)

    def evaluate(self, test_loader, batch_size=1, n_features=1):
        """The method evaluate performs the model evaluation

        The method takes DataLoaders for the test dataset, batch size for mini-batch testing,
        and number of features as inputs. Similar to the model validation, it iteratively
        predicts the target values and calculates losses. Then, it returns two lists that
        hold the predictions and the actual values.

        Note:
            This method assumes that the prediction from the previous step is available at
            the time of the prediction, and only does one-step prediction into the future.

        Args:
            test_loader (torch.utils.data.DataLoader): DataLoader that stores test data
            batch_size (int): Batch size for mini-batch training
            n_features (int): Number of feature columns

        Returns:
            list[float]: The values predicted by the model
            list[float]: The actual values in the test set.

        """
        with torch.no_grad():
            predictions = []
            values = []
            for x_test, y_test in test_loader:
                x_test = x_test.view([batch_size, -1, n_features]).to(device)
                y_test = y_test.to(device)
                self.model.eval()
                yhat = self.model(x_test)
                predictions.append(yhat.to(device).detach().numpy())
                values.append(y_test.to(device).detach().numpy())

        return predictions, values

    def plot_losses(self):
        """The method plots the calculated loss values for training and validation
        """
        plt.plot(self.train_losses, label="Training loss")
        plt.plot(self.val_losses, label="Validation loss")
        plt.legend()
        plt.title("Losses")
        plt.show()
        plt.close()

In [10]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler, MaxAbsScaler, RobustScaler

def get_scaler(scaler):
    scalers = {
        "minmax": MinMaxScaler,
        "standard": StandardScaler,
        "maxabs": MaxAbsScaler,
        "robust": RobustScaler,
    }
    return scalers.get(scaler.lower())()

In [11]:
def inverse_transform(scaler, df, columns):
    for col in columns:
        df[col] = scaler2.inverse_transform(df[col])
    return df


def format_predictions(predictions, values, df_test, scaler):
    vals = np.concatenate(values, axis=0).ravel()
    preds = np.concatenate(predictions, axis=0).ravel()
    df_result = pd.DataFrame(data={"value": vals, "prediction": preds}, index=df_test.head(len(vals)).index)
    df_result = df_result.sort_index()
    # df_result = inverse_transform(scaler, df_result, [["value", "prediction"]])
    return df_result

In [31]:
AHB_numbers = [0] # q
validate_sizes = [1] # i
test_sizes = [0.1] # j 
hidden_dims = [8] # l
layer_dims = [1] # m
dropouts = [0.2] # n
n_epochss = [1000] # o
models = ['gru'] # p
patience = 3
for q in range(len(AHB_numbers)):
    for i in range(len(validate_sizes)):
        for j in range(len(test_sizes)):
            for l in range(len(hidden_dims)):
                for m in range(len(layer_dims)):
                    for n in range(len(dropouts)):
                        for o in range(len(n_epochss)):
                            for p in range(len(models)):

                                X = AHB_data[AHB_numbers[q]].drop(['y','AHB_'+str(AHB_numbers[q]+1)], axis=1)
                                y = AHB_data[AHB_numbers[q]].drop(AHB_data[AHB_numbers[q]].columns.difference(['y']), axis=1) 

                                X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_sizes[j], train_size=1-test_sizes[j], shuffle=False, stratify=None)

                                X_val = pd.DataFrame(X_train.tail(validate_sizes[i]))
                                X_train = X_train.drop(X_train.tail(validate_sizes[i]).index)
                                y_val = pd.DataFrame(y_train.tail(validate_sizes[i]))
                                y_train = y_train.drop(y_train.tail(validate_sizes[i]).index)



                                scaler = get_scaler('minmax')
                                scaler2 = get_scaler('minmax')

                                X_train_arr = scaler.fit_transform(X_train)
                                X_val_arr = scaler.transform(X_val)

                                y_train_arr = scaler2.fit_transform(y_train)
                                y_val_arr = scaler2.transform(y_val)


                                train_features = torch.Tensor(X_train_arr)
                                train_targets = torch.Tensor(y_train_arr)
                                val_features = torch.Tensor(X_val_arr)
                                val_targets = torch.Tensor(y_val_arr)

                                train = TensorDataset(train_features, train_targets)
                                val = TensorDataset(val_features, val_targets)


                                input_dim = len(X.columns)
                                output_dim = 1
                                hidden_dim = hidden_dims[l]
                                layer_dim = layer_dims[m]
                                batch_size = 1
                                dropout = dropouts[n]
                                n_epochs = n_epochss[o]
                                learning_rate = 1e-3
                                weight_decay = 1e-6
                                patience = patience

                                model_params = {'input_dim': input_dim,
                                                'hidden_dim' : hidden_dim,
                                                'layer_dim' : layer_dim,
                                                'output_dim' : output_dim,
                                                'dropout_prob' : dropout}

                                model = get_model(models[p], model_params)

                                loss_fn = nn.MSELoss(reduction="mean")
                                optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)

                                train_loader = DataLoader(train, batch_size=5, shuffle=False, drop_last=True)
                                val_loader = DataLoader(val, batch_size=batch_size, shuffle=False, drop_last=True)

In [33]:
            for x_batch, y_batch in train_loader:
                x_batch = x_batch.view([5, -1, input_dim]).to(device)
                print(x_batch)
                #print(y_batch)

tensor([[[7.3121e-05, 0.0000e+00, 9.9239e-04, 0.0000e+00, 5.3176e-04,
          2.1631e-04, 9.5727e-02, 3.4219e-04]],

        [[7.3121e-05, 0.0000e+00, 8.2699e-04, 0.0000e+00, 4.6529e-04,
          2.1631e-04, 9.5727e-02, 3.4219e-04]],

        [[0.0000e+00, 0.0000e+00, 8.2699e-04, 0.0000e+00, 3.9882e-04,
          2.1631e-04, 9.5727e-02, 3.4219e-04]],

        [[0.0000e+00, 8.3436e-02, 8.2699e-04, 0.0000e+00, 3.3235e-05,
          2.1631e-04, 9.5727e-02, 2.2813e-04]],

        [[0.0000e+00, 8.3436e-02, 6.6159e-04, 0.0000e+00, 0.0000e+00,
          2.1631e-04, 9.5727e-02, 2.2813e-04]]])
tensor([[[1.9472e-01, 8.3436e-02, 6.6159e-04, 0.0000e+00, 2.0606e-02,
          2.1631e-04, 1.7713e-01, 2.2813e-04]],

        [[1.9472e-01, 1.8135e-01, 6.6159e-04, 0.0000e+00, 2.0938e-02,
          2.1631e-04, 1.7713e-01, 3.4219e-04]],

        [[1.9472e-01, 2.0491e-01, 4.9620e-04, 0.0000e+00, 2.0905e-02,
          2.1631e-04, 1.7713e-01, 3.4219e-04]],

        [[1.9472e-01, 2.0491e-01, 4.9620e-04, 0.

In [ ]:
%matplotlib inline
AHB_numbers = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11 , 12] # q
validate_sizes = [1] # i
test_sizes = [0.1] # j 
hidden_dims = [8, 16, 32, 64, 128] # l
layer_dims = [1, 2, 3] # m
dropouts = [0.2, 0.3, 0.5] # n
n_epochss = [1000] # o
models = ['gru', 'lstm', 'rnn'] # p
patience = 3

count = 1
performance_json = []
for q in range(len(AHB_numbers)):
    for i in range(len(validate_sizes)):
        for j in range(len(test_sizes)):
            for l in range(len(hidden_dims)):
                for m in range(len(layer_dims)):
                    for n in range(len(dropouts)):
                        for o in range(len(n_epochss)):
                            for p in range(len(models)):

                                print('PROGRESS: [ ' + str(count) + ' / ' + str(len(AHB_numbers)*len(validate_sizes)*len(test_sizes)*len(hidden_dims)*len(layer_dims)*len(dropouts)*len(n_epochss)*len(models)) +' ]')
                                
                                # Univariate
                                X = AHB_data[AHB_numbers[q]].drop(AHB_data[AHB_numbers[q]].columns.difference(['AHB_'+str(AHB_numbers[q]+1)]), axis=1) 
                                y = AHB_data[AHB_numbers[q]].drop(AHB_data[AHB_numbers[q]].columns.difference(['y']), axis=1) 

                                X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_sizes[j], train_size=1-test_sizes[j], shuffle=False, stratify=None)

                                X_val = pd.DataFrame(X_train.tail(validate_sizes[i]))
                                X_train = X_train.drop(X_train.tail(validate_sizes[i]).index)
                                y_val = pd.DataFrame(y_train.tail(validate_sizes[i]))
                                y_train = y_train.drop(y_train.tail(validate_sizes[i]).index)



                                scaler = get_scaler('minmax')
                                scaler2 = get_scaler('minmax')

                                X_train_arr = scaler.fit_transform(X_train)
                                X_val_arr = scaler.transform(X_val)

                                y_train_arr = scaler2.fit_transform(y_train)
                                y_val_arr = scaler2.transform(y_val)


                                train_features = torch.Tensor(X_train_arr)
                                train_targets = torch.Tensor(y_train_arr)
                                val_features = torch.Tensor(X_val_arr)
                                val_targets = torch.Tensor(y_val_arr)

                                train = TensorDataset(train_features, train_targets)
                                val = TensorDataset(val_features, val_targets)


                                input_dim = len(X.columns)
                                output_dim = 1
                                hidden_dim = hidden_dims[l]
                                layer_dim = layer_dims[m]
                                batch_size = 1
                                dropout = dropouts[n]
                                n_epochs = n_epochss[o]
                                learning_rate = 1e-3
                                weight_decay = 1e-6
                                patience = patience

                                model_params = {'input_dim': input_dim,
                                                'hidden_dim' : hidden_dim,
                                                'layer_dim' : layer_dim,
                                                'output_dim' : output_dim,
                                                'dropout_prob' : dropout}

                                model = get_model(models[p], model_params)

                                loss_fn = nn.MSELoss(reduction="mean")
                                optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)

                                train_loader = DataLoader(train, batch_size=batch_size, shuffle=False, drop_last=True)
                                val_loader = DataLoader(val, batch_size=batch_size, shuffle=False, drop_last=True)

                                opt = Optimization(model=model, loss_fn=loss_fn, optimizer=optimizer)
                                opt.train(train_loader, val_loader, batch_size=batch_size, n_epochs=n_epochs, n_features=input_dim, patience=patience)
                                opt.plot_losses()

                                X_test_arr = scaler.transform(X_test)
                                y_test_arr = scaler2.transform(y_test)

                                test_features = torch.Tensor(X_test_arr)
                                test_targets = torch.Tensor(y_test_arr)

                                test = TensorDataset(test_features, test_targets)

                                test_loader = DataLoader(test, batch_size=batch_size, shuffle=False, drop_last=True)
                                test_loader_one = DataLoader(test, batch_size=1, shuffle=False, drop_last=True)


                                predictions, values = opt.evaluate(test_loader_one, batch_size=1, n_features=input_dim)
                                df_result = format_predictions(predictions, values, X_test, scaler)
                                df_result_inverse = scaler2.inverse_transform(df_result)
                                df_result_inverse = pd.DataFrame(df_result_inverse, index=df_result.index, columns=df_result.columns)

                                mae = mean_absolute_error(df_result.value, df_result.prediction)
                                mse = mean_squared_error(df_result.value, df_result.prediction)
                                rmse = mean_squared_error(df_result.value, df_result.prediction, squared = False)
                                mape = mean_absolute_percentage_error(df_result.value, df_result.prediction)
                                
                                mae_inverse = mean_absolute_error(df_result_inverse.value, df_result_inverse.prediction)
                                mse_inverse = mean_squared_error(df_result_inverse.value, df_result_inverse.prediction)
                                rmse_inverse = mean_squared_error(df_result_inverse.value, df_result_inverse.prediction, squared = False)
                                mape_inverse = mean_absolute_percentage_error(df_result_inverse.value, df_result_inverse.prediction)



                                performance_json.append({'AHB_number': AHB_numbers[q]+1, 'Multi_Uni': 'Uni', 'validate_size': validate_sizes[i], \
                                                         'test_size': test_sizes[j], 'hidden_dim': hidden_dims[l], 'layer_dim':layer_dims[m], \
                                                         'dropout': dropouts[n], 'n_epochs': n_epochss[o], 'last_epoch': opt.last_epoch,'model': models[p],'MAE_normalize': float(mae), \
                                                         'MSE_normalize': float(mse), 'RMSE_normalize': float(rmse), 'MAPE_normalize': float(mape), \
                                                         'MAE': float(mae_inverse), 'MSE': float(mse_inverse), 'RMSE': float(rmse_inverse), 'MAPE': float(mape_inverse)})
                                

                                json_formatted_str = json.dumps(performance_json[-1], indent=2)
                                print(json_formatted_str)



                                # Multivariate
                                X = AHB_data[AHB_numbers[q]].drop(['y','AHB_'+str(AHB_numbers[q]+1)], axis=1)
                                y = AHB_data[AHB_numbers[q]].drop(AHB_data[AHB_numbers[q]].columns.difference(['y']), axis=1) 

                                X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_sizes[j], train_size=1-test_sizes[j], shuffle=False, stratify=None)

                                X_val = pd.DataFrame(X_train.tail(validate_sizes[i]))
                                X_train = X_train.drop(X_train.tail(validate_sizes[i]).index)
                                y_val = pd.DataFrame(y_train.tail(validate_sizes[i]))
                                y_train = y_train.drop(y_train.tail(validate_sizes[i]).index)



                                scaler = get_scaler('minmax')
                                scaler2 = get_scaler('minmax')

                                X_train_arr = scaler.fit_transform(X_train)
                                X_val_arr = scaler.transform(X_val)

                                y_train_arr = scaler2.fit_transform(y_train)
                                y_val_arr = scaler2.transform(y_val)


                                train_features = torch.Tensor(X_train_arr)
                                train_targets = torch.Tensor(y_train_arr)
                                val_features = torch.Tensor(X_val_arr)
                                val_targets = torch.Tensor(y_val_arr)

                                train = TensorDataset(train_features, train_targets)
                                val = TensorDataset(val_features, val_targets)


                                input_dim = len(X.columns)
                                output_dim = 1
                                hidden_dim = hidden_dims[l]
                                layer_dim = layer_dims[m]
                                batch_size = 1
                                dropout = dropouts[n]
                                n_epochs = n_epochss[o]
                                learning_rate = 1e-3
                                weight_decay = 1e-6
                                patience = patience

                                model_params = {'input_dim': input_dim,
                                                'hidden_dim' : hidden_dim,
                                                'layer_dim' : layer_dim,
                                                'output_dim' : output_dim,
                                                'dropout_prob' : dropout}

                                model = get_model(models[p], model_params)

                                loss_fn = nn.MSELoss(reduction="mean")
                                optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)

                                train_loader = DataLoader(train, batch_size=batch_size, shuffle=False, drop_last=True)
                                val_loader = DataLoader(val, batch_size=batch_size, shuffle=False, drop_last=True)

                                opt = Optimization(model=model, loss_fn=loss_fn, optimizer=optimizer)
                                opt.train(train_loader, val_loader, batch_size=batch_size, n_epochs=n_epochs, n_features=input_dim, patience=patience)
                                opt.plot_losses()

                                X_test_arr = scaler.transform(X_test)
                                y_test_arr = scaler2.transform(y_test)

                                test_features = torch.Tensor(X_test_arr)
                                test_targets = torch.Tensor(y_test_arr)

                                test = TensorDataset(test_features, test_targets)

                                test_loader = DataLoader(test, batch_size=batch_size, shuffle=False, drop_last=True)
                                test_loader_one = DataLoader(test, batch_size=1, shuffle=False, drop_last=True)


                                predictions, values = opt.evaluate(test_loader_one, batch_size=1, n_features=input_dim)
                                df_result = format_predictions(predictions, values, X_test, scaler)
                                df_result_inverse = scaler2.inverse_transform(df_result)
                                df_result_inverse = pd.DataFrame(df_result_inverse, index=df_result.index, columns=df_result.columns)

                                mae = mean_absolute_error(df_result.value, df_result.prediction)
                                mse = mean_squared_error(df_result.value, df_result.prediction)
                                rmse = mean_squared_error(df_result.value, df_result.prediction, squared = False)
                                mape = mean_absolute_percentage_error(df_result.value, df_result.prediction)
                                
                                mae_inverse = mean_absolute_error(df_result_inverse.value, df_result_inverse.prediction)
                                mse_inverse = mean_squared_error(df_result_inverse.value, df_result_inverse.prediction)
                                rmse_inverse = mean_squared_error(df_result_inverse.value, df_result_inverse.prediction, squared = False)
                                mape_inverse = mean_absolute_percentage_error(df_result_inverse.value, df_result_inverse.prediction)



                                performance_json.append({'AHB_number': AHB_numbers[q]+1, 'Multi_Uni': 'Multi', 'validate_size': validate_sizes[i], \
                                                         'test_size': test_sizes[j], 'hidden_dim': hidden_dims[l], 'layer_dim':layer_dims[m], \
                                                         'dropout': dropouts[n], 'n_epochs': n_epochss[o], 'last_epoch': opt.last_epoch,'model': models[p],'MAE_normalize': float(mae), \
                                                         'MSE_normalize': float(mse), 'RMSE_normalize': float(rmse), 'MAPE_normalize': float(mape), \
                                                         'MAE': float(mae_inverse), 'MSE': float(mse_inverse), 'RMSE': float(rmse_inverse), 'MAPE': float(mape_inverse)})
                                

                                json_formatted_str = json.dumps(performance_json[-1], indent=2)
                                print(json_formatted_str)
                                count = count + 1

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
json.dump(performance_json, open("all_performance_jsonno_no_cross_val.json","w"))
df_json = pd.read_json('/content/all_performance_jsonno_no_cross_val.json')
df_json.to_csv('/content/all_performance_json_no_cross_val.csv', index=False)
! cp /content/all_performance_json_no_cross_val.json /content/drive/MyDrive/Depression
! cp /content/all_performance_json_no_cross_val.csv /content/drive/MyDrive/Depression

cp: cannot stat '/content/all_performance_json_no_cross_val.json': No such file or directory


In [ ]:
df_json

,AHB_number,Multi_Uni,validate_size,test_size,hidden_dim,layer_dim,dropout,n_epochs,last_epoch,model,MAE_normalize,MSE_normalize,RMSE_normalize,MAPE_normalize,MAE,MSE,RMSE,MAPE
0,1,Uni,1,0.1,8,1,0.2,1000,113,gru,0.012597,0.000195,0.013967,0.011678,1008.799133,1.250944e+06,1118.456299,0.007203
1,1,Multi,1,0.1,8,1,0.2,1000,232,gru,0.003308,0.000016,0.003993,0.003074,264.906250,1.022384e+05,319.747467,0.001894
2,1,Uni,1,0.1,8,1,0.2,1000,24,lstm,0.015367,0.000285,0.016895,0.014241,1230.573608,1.830430e+06,1352.933960,0.008785
3,1,Multi,1,0.1,8,1,0.2,1000,12,lstm,0.046470,0.002270,0.047639,0.043272,3721.303467,1.455400e+07,3814.970459,0.026643
4,1,Uni,1,0.1,8,1,0.2,1000,56,rnn,0.010015,0.000133,0.011538,0.009270,801.986633,8.536719e+05,923.943665,0.005721
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3505,13,Multi,1,0.1,128,3,0.5,1000,5,gru,0.136841,0.019606,0.140020,0.127863,2166.877686,4.916034e+06,2217.213135,0.053911
3506,13,Uni,1,0.1,128,3,0.5,1000,27,lstm,0.085525,0.008024,0.089574,0.079134,1354.288452,2.011872e+06,1418.404663,0.033555
3507,13,Multi,1,0.1,128,3,0.5,1000,86,lstm,0.074214,0.006258,0.079108,0.068381,1175.177490,1.569210e+06,1252.681030,0.029066
3508,13,Uni,1,0.1,128,3,0.5,1000,14,rnn,0.119502,0.015163,0.123140,0.110432,1892.311890,3.802192e+06,1949.921143,0.046860


In [ ]:
df_json_gru = df_json[df_json['model'] == 'gru']
df_json_gru[df_json_gru['MAE_normalize'] == df_json_gru['MAE_normalize'].min()]

,AHB_number,Multi_Uni,validate_size,test_size,hidden_dim,layer_dim,dropout,n_epochs,last_epoch,model,MAE_normalize,MSE_normalize,RMSE_normalize,MAPE_normalize,MAE,MSE,RMSE,MAPE
2538,10,Uni,1,0.1,32,1,0.2,1000,15,gru,0.002984,0.000012,0.003439,0.002819,153.560272,31333.800781,177.013565,0.001501


In [ ]:
df_json_lstm = df_json[df_json['model'] == 'lstm']
df_json_lstm[df_json_lstm['MAE_normalize'] == df_json_lstm['MAE_normalize'].min()]

,AHB_number,Multi_Uni,validate_size,test_size,hidden_dim,layer_dim,dropout,n_epochs,last_epoch,model,MAE_normalize,MSE_normalize,RMSE_normalize,MAPE_normalize,MAE,MSE,RMSE,MAPE
759,3,Multi,1,0.1,128,1,0.2,1000,16,lstm,0.002838,0.000015,0.003904,0.00268,103.029015,20087.087891,141.728928,0.001512


In [ ]:
df_json_rnn = df_json[df_json['model'] == 'rnn']
df_json_rnn[df_json_rnn['MAE_normalize'] == df_json_rnn['MAE_normalize'].min()]

,AHB_number,Multi_Uni,validate_size,test_size,hidden_dim,layer_dim,dropout,n_epochs,last_epoch,model,MAE_normalize,MSE_normalize,RMSE_normalize,MAPE_normalize,MAE,MSE,RMSE,MAPE
2608,10,Uni,1,0.1,64,1,0.5,1000,13,rnn,0.00276,0.000011,0.003373,0.002613,142.059158,30137.537109,173.601669,0.00139
